# Aimsun SQLite Output Database Visualize

### Goal of the notebook

This purpose of this Jupyter notebook is to visualize the database produced by the Aimsun simulations.
***
**Outputs:** 

Interactive Kepler GL web apps

**Inputs:** 

Aimsun simulation outputs:
- aimsun-outputs.sqlite
- aimsun_outputs.csv

Ground truth data:
- ground.sqlite
- ground.csv

Aimsun shape files:
- nodes
- detectors
- sections

**Dependent scripts:** 
None

 **IMPORTANT:** Run the iPython cell below in order to import the necessary packages.

In [1]:
# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fremontdropbox import get_dropbox_location

path_dropbox = get_dropbox_location()
data_folder = os.path.join(path_dropbox, 'Private Structured data collection')
sql_folder = os.path.join(data_folder, 'Aimsun','Outputs')
input_folder = os.path.join(data_folder, 'Aimsun','Inputs')

In [2]:
import numpy as np
import scipy
import matplotlib
import sqlite3
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from keplergl import KeplerGl

### SQLite Helper Functions

These functions are wrapper functions for the underlying SQLite commands we use to query data from the SQLite database. The notebook assumes that the Aimsun database file has been placed into the current directory of the notebook.

In [3]:
def create_connection(db_file):
    conn = sqlite3.connect(db_file)
 
    return conn

def select_all_from_table(conn, table, should_print = True):
    cur = conn.cursor()
    
    if should_print:
        # Prevents us from accidentally clogging up the notebook with huge print statements
        query = "SELECT * FROM {} LIMIT 10".format(table)
    else:
        query = "SELECT * FROM {}".format(table)

    if should_print:
        for row in rows:
            print(row)
            
    cur.execute(query)
    
    results = pd.DataFrame.from_records(data = cur.fetchall())
    results.columns = list(map(lambda x: x[0], cur.description))
    return results

def select_where_from_table(conn, table, clause, should_print = True):
    cur = conn.cursor()
    cur.execute("SELECT * FROM {} WHERE {}".format(table, clauses))
 
    rows = cur.fetchall()
 
    results = pd.DataFrame.from_records(data = cur.fetchall())
    results.columns = list(map(lambda x: x[0], cur.description))
    return results

def show_all_tables(conn, should_print = True):
    cur = conn.cursor()
    cur.execute('SELECT name from sqlite_master where type= "table"')
 
    rows = cur.fetchall()
 
    results = pd.DataFrame.from_records(data = cur.fetchall())
    results.columns = list(map(lambda x: x[0], cur.description))
    return results

### Main AimsunAnalyzer Class

To be implemented. The skeleton code has been given below.

In [10]:
class AimsunAnalyzer:
    
    node_cols = ['oid', 'sid', 'ent', 'approachDelay', 'lostVehicles', 'missedTurnings']
    detector_cols = ['oid', 'sid', 'ent', 'countveh', 'flow', 'speed', 'density', 'occupancy', 'headway']
    lane_cols = ['oid', 'eid', 'sid', 'ent', 'lane', 'count', 'flow','input_count', 'input_flow',
       'density', 'qmean', 'qmax', 'dtime', 'wtimeVQ', 'speed', 'hspeed', 'ttime', 'dtimeTtime', 'stime']
    
    detector_in_cols = ['id', 'name', 'section_id', 'position', 'length', 'from_lane', 'to_lane', 'geometry']
    node_in_cols = ['eid', 'nodetype', 'geometry']
    sections_in_cols = ['id', 'eid', 'name', 'nb_lanes', 'speed', 'capacity', 'rd_type', 'func_class', 'fnode', 'tnode', 'geometry']
    
    def __init__(self, simulation_file, simulation_filetype, ground_truth_file = None, ground_truth_filetype = None):
        """
        Initializes the Aimsun analyzer.
        
        @param simulation_file:          The file path of the source file of Aimsun macro/microsimulation outputs.
        @param simulation_filetype:      The type of the src_simulation file (can be .csv or .sqlite).
        @param ground_truth_file:        The file path of the source file of Aimsun macro/microsimulation outputs.
        @param ground_truth_filetype:    The type of the src_simulation file (can be .csv or .sqlite).
        """
        self.database = simulation_file
        self.conn = create_connection(self.database)
        print("=====Connection Established.=====")
        
        self.nodes = select_all_from_table(self.conn, "MINODE", should_print = False)[self.node_cols]
        self.detectors = select_all_from_table(self.conn, "MIDETEC", should_print = False)[self.detector_cols]
        self.lanes = select_all_from_table(self.conn, "MILANE", should_print = False)[self.lane_cols]
        
        self.nodes_inputs = gpd.read_file(input_folder + "/nodes.shp")[self.node_in_cols]
        self.detector_inputs = gpd.read_file(input_folder + "/detectors.shp")[self.detector_in_cols]
        self.section_inputs = gpd.read_file(input_folder + "/sections.shp")
        
        self.nodes_inputs['eid'] = self.nodes_inputs['eid'].fillna(0)
        self.nodes_inputs['eid'] = self.nodes_inputs['eid'].astype('int64')
        
        # print(self.section_inputs)
        # print(self.lanes)
        self.nodes_joined = self.nodes_inputs.merge(self.nodes, how = 'left', left_on = 'eid', right_on = 'oid')
        # self.section_joined = self.section_inputs.merge(self.lanes, how = 'left', on = 'eid')
        self.detector_joined = self.detector_inputs.merge(self.detectors, how = 'left', left_on = 'id', right_on = 'oid')
            
        print("=====Simulation Data Loaded.=====")
        
        self.ground_truth_file = ground_truth_file
        self.ground_truth_filetype = ground_truth_filetype
    
    def visualize_node_data(self, columns, time = 10):
        for column in columns:
            if column not in self.nodes_joined:
                print("Error: {} not found in column".format(column))
                return None
        
        columns = ['geometry'] + columns
        selected_data = self.nodes_joined[self.nodes_joined['ent'] == time][columns]
        
        kepler_map = KeplerGl(height=500)
        kepler_map.add_data(data=selected_data, name='Section {}'.format(','.join([c for c in columns])))
        return kepler_map
    
    def visualize_section_data(self, columns, time = 10):
        for column in columns:
            if column not in self.section_joined:
                print("Error: {} not found in column".format(column))
                return None
        
        columns = ['geometry'] + columns
        selected_data = self.section_joined[self.section_joined['ent'] == time][columns]
        
        kepler_map = KeplerGl(height=500)
        kepler_map.add_data(data=selected_data, name='Section {}'.format(','.join([c for c in columns])))
        return kepler_map

    def visualize_detector_data(self, columns, time = 10):
        for column in columns:
            if column not in self.detector_joined:
                print("Error: {} not found in column".format(column))
                return None
        
        columns = ['geometry'] + columns
        selected_data = self.detector_joined[self.detector_joined['ent'] == time][columns]
        
        kepler_map = KeplerGl(height=500)
        kepler_map.add_data(data=selected_data, name='Section {}'.format(','.join([c for c in columns])))
        return kepler_map

## Main Code

This section contains the code that should be run in order to generate results in this notebook. Below is the necessary starter code that connects to the database and outputs the current simulator info:

In [11]:
database = os.path.join(sql_folder, "2017.sqlite")
analyzer = AimsunAnalyzer(database, "sqlite")

=====Connection Established.=====
=====Simulation Data Loaded.=====


### Visualizer Code

In [12]:
#analyzer.lanes[(analyzer.lanes['eid'] == '242') & (analyzer.lanes['sid'] == 1)]
#analyzer.lanes[(analyzer.lanes['count'] != 0) & (analyzer.lanes['eid'] == '7732')]
analyzer.visualize_node_data([])

KeplerGl(data={'Section geometry': {'index': [18, 19, 92, 93, 166, 167, 240, 241, 314, 315, 388, 389, 462, 463…

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


In [ ]:
analyzer.visualize_section_data(['speed_x', 'speed_y'])